# day 8

https://adventofcode.com/2021/day/8

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day08.txt')

LOGGER = logging.getLogger('day08')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read()

In [ ]:
def parse_data(d):
    x = []
    for line in d.strip().split('\n'):
        signal_patterns, output_vals = line.split(' | ')
        signal_patterns = signal_patterns.split(' ')
        output_vals = output_vals.split(' ')
        x.append((signal_patterns, output_vals))
    return x

#### function def

In [ ]:
def q_1(data):
    special_lens = [2, 3, 4, 7]
    special_items = [v
                     for (signal_patterns, output_vals) in parse_data(data)
                     for v in output_vals
                     if len(v) in special_lens]
    return len(special_items)

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 26
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
from itertools import permutations

In [ ]:
VALID_SEGMENTS = [
    'abcefg',   # 0
    'cf',       # 1
    'acdeg',    # 2
    'acdfg',    # 3
    'bcdf',     # 4
    'abdfg',    # 5
    'abdefg',   # 6
    'acf',      # 7
    'abcdefg',  # 8
    'abcdfg',   # 9
]
VALID_SEGMENTS = [sorted(_) for _ in VALID_SEGMENTS]

In [ ]:
SEGMENTS = 'abcdefg'

In [ ]:
def is_valid_map(m, wires_list):
    """given a mapping from wire to segment `m` and an iterable
    of active wires `vals`, verify that the activated segments make
    a valid number"""
    return all(wires_to_segments(m, wires) in VALID_SEGMENTS
               for wires in wires_list)

test_m = dict(zip('deafgbc', SEGMENTS))
test_m_fail = dict(zip('edafgbc', SEGMENTS))
test_wires_list = [
    'acedgfb',
    'cdfbe',
    'gcdfa',
    'fbcad',
    'dab',
    'cefabd',
    'cdfgeb',
    'eafb',
    'cagedb',
    'ab',
]
assert is_valid_map(test_m, test_wires_list)
assert not is_valid_map(test_m_fail, test_wires_list)

In [ ]:
def wires_to_segments(m, wires):
    """given a mapping from wire to segment `m` and a string
    representing active wires, return the corresponding active
    segments"""
    return sorted(m[w] for w in wires)

test_m = dict(zip('deafgbc', SEGMENTS))
assert wires_to_segments(test_m, 'acedgfb') == VALID_SEGMENTS[8]
assert wires_to_segments(test_m, 'cdfbe') == VALID_SEGMENTS[5]
assert wires_to_segments(test_m, 'gcdfa') == VALID_SEGMENTS[2]
assert wires_to_segments(test_m, 'fbcad') == VALID_SEGMENTS[3]
assert wires_to_segments(test_m, 'dab') == VALID_SEGMENTS[7]
assert wires_to_segments(test_m, 'cefabd') == VALID_SEGMENTS[9]
assert wires_to_segments(test_m, 'cdfgeb') == VALID_SEGMENTS[6]
assert wires_to_segments(test_m, 'eafb') == VALID_SEGMENTS[4]
assert wires_to_segments(test_m, 'cagedb') == VALID_SEGMENTS[0]
assert wires_to_segments(test_m, 'ab') == VALID_SEGMENTS[1]

In [ ]:
def wires_to_ints(m, vals):
    return [VALID_SEGMENTS.index(wires_to_segments(m, val))
            for val in vals]

test_m = dict(zip('deafgbc', SEGMENTS))
test_vals = [
    'cdfeb',
    'fcadb',
    'cdfeb',
    'cdbaf',
]
assert wires_to_ints(test_m, test_vals) == [5, 3, 5, 3]

In [ ]:
def q_2(data):
    parsed_output_vals = []
    for (signal_patterns, output_vals) in parse_data(data):
        # re-order signal_patterns to go from shortest to longest
        # because there are so few valid 2, 3, and 4 element maps
        signal_patterns = sorted(signal_patterns, key=lambda l: len(l))
        
        for wire_perm in permutations(SEGMENTS, 7):
            wire_segment_map = dict(zip(wire_perm, SEGMENTS))
            
            if is_valid_map(wire_segment_map, signal_patterns):
                parsed_output_vals.append(wires_to_ints(wire_segment_map, output_vals))
    
    # turn the parsed vals into ints
    parsed_output_vals = [(1_000 * a + 100 * b + 10 * c + d)
                          for (a, b, c, d) in parsed_output_vals]
    return sum(parsed_output_vals)

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 61229
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin